In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/imdb-noiseddata'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        imdb_data =  pd.read_csv(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-noiseddata/IMDB_denoisedData.csv


In [2]:
imdb_dataset_v1 = imdb_data[['random_augumented','review']]

In [3]:
imdb_dataset_v1.columns

Index(['random_augumented', 'review'], dtype='object')

In [6]:
imdb_dataset_v1 = imdb_dataset_v1.rename(columns={"review": "solution", "random_augumented": "original"}, errors="raise")

In [7]:
imdb_dataset_v1.columns

Index(['original', 'solution'], dtype='object')

In [8]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

from tqdm import tqdm
import time
import random

import torch
from transformers import EncoderDecoderModel, AutoTokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [9]:
seed_val = 66
MIN_LEN = 20
MAX_LEN = 300
batch_size = 6
EPOCHS = 25
early_stopping_rounds = 5
lr = 5e-5

In [10]:
torch.cuda.empty_cache()

In [11]:
imdb_dataset_v1['original'] = imdb_dataset_v1['original'].map(lambda x: x.replace('<br /> <br />',""))

In [12]:
imdb_dataset_v1['solution'] = imdb_dataset_v1['solution'].map(lambda x: x.replace('<br /><br />',""))

In [13]:
imdb_dataset_v1.head(-1)

,original,solution
0,One of the other reviewers has mentioned that ...,One of the other reviewers has mentioned that ...
1,A wonderful little production. The filming te...,A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...,I thought this was a wonderful way to spend ti...
3,Basically there ' s a family where a little bo...,Basically there's a family where a little boy ...
4,"Petter Mattei ' s "" Love in the Time of Money ...","Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...
49994,This is your typical junk comedy. There are a...,This is your typical junk comedy.There are alm...
49995,I thought this movie did a down right gooid jo...,I thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad act&ing, idiotic d...","Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in p1aerochial elementa...,I am a Catholic taught in parochial elementary...


In [14]:
train = imdb_dataset_v1[:15000]

In [15]:
train.head(-1)

,original,solution
0,One of the other reviewers has mentioned that ...,One of the other reviewers has mentioned that ...
1,A wonderful little production. The filming te...,A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...,I thought this was a wonderful way to spend ti...
3,Basically there ' s a family where a little bo...,Basically there's a family where a little boy ...
4,"Petter Mattei ' s "" Love in the Time of Money ...","Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...
14994,* * * out of * * * * Yep! Dressed To Kill is ...,*** out of ****Yep! Dressed To Kill is that ki...
14995,Bobcat Goldthwait should be commended for atte...,Bobcat Goldthwait should be commended for atte...
14996,"And it ' s not because since her days on "" ACl...","And it's not because since her days on ""Claris..."
14997,A traveling couple (Horton and Hamilton) stumb...,A traveling couple (Horton and Hamilton)stumbl...


In [16]:
train.original.apply(lambda x: len(x.split())).describe()

count    15000.000000
mean       249.054200
std        185.249128
min          9.000000
25%        135.000000
50%        186.000000
75%        304.000000
max       1959.000000
Name: original, dtype: float64

In [17]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [18]:
kf = KFold(n_splits=5)

for train_index, val_index in kf.split(train):
    break

In [19]:
val = train.iloc[val_index]
train = train.iloc[train_index]

In [20]:
trainX = torch.Tensor(np.asarray([tokenizer.encode(i, max_length=MAX_LEN, truncation=True, padding='max_length', add_special_tokens=True) \
                                  for i in tqdm(train.original.values)]))
trainy = torch.Tensor(np.asarray([tokenizer.encode(i, max_length=MAX_LEN, truncation=True, padding='max_length', add_special_tokens=True) \
                                  for i in tqdm(train.solution.values)]))

valX = torch.Tensor(np.asarray([tokenizer.encode(i, max_length=MAX_LEN, truncation=True, padding='max_length', add_special_tokens=True) \
                                for i in tqdm(val.original.values)]))
valy = torch.Tensor(np.asarray([tokenizer.encode(i, max_length=MAX_LEN, truncation=True, padding='max_length', add_special_tokens=True) \
                                for i in tqdm(val.solution.values)]))


100%|██████████| 3000/3000 [00:22<00:00, 133.44it/s]


In [21]:
trainX.shape, trainy.shape, valX.shape, valy.shape

(torch.Size([12000, 300]),
 torch.Size([12000, 300]),
 torch.Size([3000, 300]),
 torch.Size([3000, 300]))

In [22]:
trainX = torch.tensor(trainX, dtype=torch.long)
trainy = torch.tensor(trainy, dtype=torch.long)
valX = torch.tensor(valX, dtype=torch.long)
valy = torch.tensor(valy, dtype=torch.long)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy 

In [23]:
for dirname, _, filenames in os.walk('/kaggle/input/bertdenioseimdb'):
    for filename in filenames:
        print (os.path.join(dirname, filename))

/kaggle/input/bertdenioseimdb/__results__.html
/kaggle/input/bertdenioseimdb/__notebook__.ipynb
/kaggle/input/bertdenioseimdb/__output__.json
/kaggle/input/bertdenioseimdb/custom.css
/kaggle/input/bertdenioseimdb/BERT_GPT2_denoise/config.json
/kaggle/input/bertdenioseimdb/BERT_GPT2_denoise/pytorch_model.bin


In [ ]:
../input/bertdenioseimdb/BERT_GPT2_denoise/config.json
../input/bertdenioseimdb

In [24]:
model = EncoderDecoderModel.from_pretrained('/kaggle/input/bertdenioseimdb//BERT_GPT2_denoise')

In [25]:
print ("Modeling")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("Device: {}".format(device))

model.to(device)

Modeling
Device: cuda


EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [26]:
test_index = 0
generated = model.generate((valX[test_index:test_index+1]).to(device), decoder_start_token_id=model.config.decoder.pad_token_id, max_length=120)
print ("Noisy text: {}".format(val.original.iloc[test_index]))
print("--------------------------------------")
print ("Original clean text: {}".format(val.solution.iloc[test_index]))
print("--------------------------------------")
print ("Predicted clean test: {}".format((" ".join([tokenizer.decode(i) for i in generated.cpu().numpy()][0].split()[1:]))))

Noisy text: One of the other reviewers has mentioned that after watching just 1 Oz episode you ' ll be hooked. They are right, as this is exactly what happened w4ith me.  The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.  It is called OZ as tihat is the nickname cgiven to the Os#wald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is h@ome to many. . Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more. .. . so scuffles, death stares, dodgy adealingKs and shady agreements are never far away.  I would say the main app@eal of the show is due to the fact t

In [27]:
test_index = 1
generated = model.generate((valX[test_index:test_index+1]).to(device), decoder_start_token_id=model.config.decoder.pad_token_id, max_length=200)
print ("Noisy text: {}".format(val.original.iloc[test_index]))
print("--------------------------------------")
print ("Original clean text: {}".format(val.solution.iloc[test_index]))
print("--------------------------------------")
print ("Predicted clean test: {}".format((" ".join([tokenizer.decode(i) for i in generated.cpu().numpy()][0].split()[1:]))))

Noisy text: A wonderful little production.  The filming technique is very unassuming - verEy old - time - BBC fashion and gives a comforting, and sometimes discomforting, sentse of realism to the entire Upiece.  The actors are extremely well chosen - Michael Sheen not only " has got all the 3polari " but he has all the voices down pat too! You can truly see the EsXeamless editing guided by the references to Williams ' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful pro)duccztion about one of the grqeat master ' s of comedy and his life.  The realism really comes home with the little things: the fantasy of the guard which, rather than use the trRa)diti@onal ' drteam ' techniques remains solid then disappears. It plays on our knowledge and our senses, sp$articulajrly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell ' s murals decorating every surface) are terribly w

In [28]:
%%time

test_index = 0
generated = model.generate((valX[test_index:test_index+1]).to(device), decoder_start_token_id=model.config.decoder.pad_token_id, max_length=150, num_beams=3)

CPU times: user 3.91 s, sys: 9.74 ms, total: 3.92 s
Wall time: 3.95 s


In [29]:
print ("Predicted clean test: {}".format((" ".join([tokenizer.decode(i) for i in generated.cpu().numpy()][0].split()[1:]))))

Predicted clean test: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me. The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word. It is called OZ as that is the nickname given to the Lone Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inward


In [30]:
%%time

test_index = 0
generated = model.generate((valX[test_index:test_index+5]).to(device), decoder_start_token_id=model.config.decoder.pad_token_id, max_length=150)

CPU times: user 5.34 s, sys: 22.8 ms, total: 5.36 s
Wall time: 5.38 s


In [31]:
def generate_text(test_index):
    generated = model.generate((valX[test_index:test_index+1]).to(device), decoder_start_token_id=model.config.decoder.pad_token_id, max_length=100)
    gen_text = [tokenizer.decode(i) for i in generated.cpu().numpy()][0]
    gen_text = gen_text.replace('[CLS]','')
    gen_text = gen_text.replace('[SEP]','')
    gen_text = gen_text.replace('[PAD]','')
    gen_text = gen_text.strip()
    
    return gen_text

In [32]:
val_100 = val[val.original.apply(lambda x: len(x.split()) <= 100)]
print (val.shape, val_100.shape)

(3000, 2) (334, 2)


In [33]:
val_100.index

Int64Index([   9,   10,   14,   22,   46,   53,   62,   63,   84,  112,
            ...
            2919, 2921, 2927, 2943, 2959, 2963, 2966, 2987, 2993, 2999],
           dtype='int64', length=334)

In [34]:
val_100.head(1)

,original,solution
9,If you likxe original gut wrenchai3ng laughter...,If you like original gut wrenching laughter yo...


In [35]:
output = []

for index, i in tqdm(enumerate(val_100.index)):
    output.append(generate_text(i))

334it [10:16,  1.85s/it]


In [36]:
output[1]

"[UNK] Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines. At first it was very odd and silly funny but as the movie progressed I didn't find the jokes or oddness funny anymore. Its a low budget film ( thats never a problem in itself ), there were some pretty interesting characters, but eventually I just lost interest. I imagine this film would appeal to a stoner who is currently naked."

In [37]:
val_100.predicted.iloc[1]

AttributeError: 'DataFrame' object has no attribute 'predicted'

In [38]:
type(val_100)

pandas.core.frame.DataFrame

In [39]:
val_100['predicted_text'] = output

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
val_100.head(-1)

,original,solution,predicted_text
9,If you likxe original gut wrenchai3ng laughter...,If you like original gut wrenching laughter yo...,[UNK] If you like original gut wrench reading ...
10,Phil the Alien is one of those quirky films wh...,Phil the Alien is one of those quirky films wh...,[UNK] Phil the Alien is one of those quirky fi...
14,This a fantastic movie of thrxee prisoners who...,This a fantastic movie of three prisoners who ...,[UNK] This a fantastic movie of three prisoner...
22,"What an absolutely sDtuWnning movie, if you ha...","What an absolutely stunning movie, if you have...","[UNK] What an absolutely stunning movie, if yo..."
46,Protocol is an implausible movie whose only sa...,Protocol is an implausible movie whose only sa...,[UNK] 1830 is an implausible movie whose only ...
...,...,...,...
2959,&This dumb comedy really does a good job of wa...,This dumb comedy really does a good job of was...,[UNK] This dumb comedy really does a good job ...
2963,"I reneted this mxovie to get an easy, entertai...","I rented this movie to get an easy, entertaine...","[UNK] I rented this movie to get an easy, ente..."
2966,"Why? !! This was an insipid, unzin9sp&ired and...","Why?!! This was an insipid, uninspired and emb...","[UNK] Why?!! This was an insipid, unbelievable..."
2987,Dream Quest was a surprisingly good movie. The...,Dream Quest was a surprisingly good movie. The...,[UNK] Dream Quest was a surprisingly good movi...


In [41]:
import numpy as np
import  nltk.translate.bleu_score as bleu

In [42]:
def WRR(text1,text2):
    a = set(text1.lower().split())
    b = set(text2.lower().split())
    
    if (len(a) == 0) and (len(b) == 0):
        return .5
    
    c = a.intersection(b)
    return float(len(c))/(len(a) + len(b) - len(c))

def levenshtein(seq1, seq2):
    seq1 = seq1.lower()
    seq2 = seq2.lower()
    
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

def CRR(text1, text2):
    try:
        return 1 - float(levenshtein(text1,text2))/max(len(text1),len(text2))
    except:
        return 0

def bleu_score(text1,text2):
    return bleu.sentence_bleu([text1.lower().split()],text2.lower().split())

In [43]:
val_1000 = val.iloc[:val_100.shape[0]]


In [44]:
val_1000.head(-1)

,original,solution
0,One of the other reviewers has mentioned that ...,One of the other reviewers has mentioned that ...
1,A wonderful little production. The filming te...,A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...,I thought this was a wonderful way to spend ti...
3,Basically there ' s a family where a little bo...,Basically there's a family where a little boy ...
4,"Petter Mattei ' s "" Love in the Time of Money ...","Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...
328,The sexploitation movie era of the late sixtie...,The sexploitation movie era of the late sixtie...
329,Whaet a surpriasinzgTly good movie this one tu...,What a surprisingly good movie this one turned...
330,"I did not expect mulch from thyis film, but bo...","I did not expect much from this film, but boy-..."
331,I cajnt believe there are peUople out tMhere t...,I cant believe there are people out there that...


In [45]:
val_100.head(-1)

,original,solution,predicted_text
9,If you likxe original gut wrenchai3ng laughter...,If you like original gut wrenching laughter yo...,[UNK] If you like original gut wrench reading ...
10,Phil the Alien is one of those quirky films wh...,Phil the Alien is one of those quirky films wh...,[UNK] Phil the Alien is one of those quirky fi...
14,This a fantastic movie of thrxee prisoners who...,This a fantastic movie of three prisoners who ...,[UNK] This a fantastic movie of three prisoner...
22,"What an absolutely sDtuWnning movie, if you ha...","What an absolutely stunning movie, if you have...","[UNK] What an absolutely stunning movie, if yo..."
46,Protocol is an implausible movie whose only sa...,Protocol is an implausible movie whose only sa...,[UNK] 1830 is an implausible movie whose only ...
...,...,...,...
2959,&This dumb comedy really does a good job of wa...,This dumb comedy really does a good job of was...,[UNK] This dumb comedy really does a good job ...
2963,"I reneted this mxovie to get an easy, entertai...","I rented this movie to get an easy, entertaine...","[UNK] I rented this movie to get an easy, ente..."
2966,"Why? !! This was an insipid, unzin9sp&ired and...","Why?!! This was an insipid, uninspired and emb...","[UNK] Why?!! This was an insipid, unbelievable..."
2987,Dream Quest was a surprisingly good movie. The...,Dream Quest was a surprisingly good movie. The...,[UNK] Dream Quest was a surprisingly good movi...


In [46]:
#val_100['predicted_text'] = l

In [47]:
val_100['WRR_1'] = val_100.apply(lambda x: WRR(x.original, x.solution), axis=1)
val_100['WRR_2'] = val_100.apply(lambda x: WRR((x.predicted_text).replace('[UNK]',""), x.solution), axis=1)

#val['CRR_1'] = val.apply(lambda x: CRR(x.original, x.solution), axis=1)
#val['CRR_2'] = val.apply(lambda x: CRR(x.predicted_text, x.solution), axis=1)

val_100['BLEU_1'] = val_100.apply(lambda x: bleu_score(x.original, x.solution), axis=1)
val_100['BLEU_2'] = val_100.apply(lambda x: bleu_score((x.predicted_text).replace('[UNK]',""), x.solution), axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python

In [49]:
val_100[['WRR_1', 'WRR_2', 'BLEU_1', 'BLEU_2']].describe()

,WRR_1,WRR_2,BLEU_1,BLEU_2
count,334.000000,334.000000,334.000000,334.000000
mean,0.590840,0.768578,0.514889,0.748420
std,0.086034,0.110935,0.116168,0.129724
min,0.275862,0.471264,0.142008,0.323858
25%,0.534483,0.685972,0.439922,0.654252
50%,0.600000,0.784200,0.527092,0.770204
75%,0.654960,0.849829,0.598135,0.845821
max,0.772152,1.000000,0.780350,1.000000


In [50]:
val_100.head(50)

,original,solution,predicted_text,WRR_1,WRR_2,BLEU_1,BLEU_2
9,If you likxe original gut wrenchai3ng laughter...,If you like original gut wrenching laughter yo...,[UNK] If you like original gut wrench reading ...,0.432432,0.785714,0.327172,0.798863
10,Phil the Alien is one of those quirky films wh...,Phil the Alien is one of those quirky films wh...,[UNK] Phil the Alien is one of those quirky fi...,0.604167,0.698795,0.492658,0.705231
14,This a fantastic movie of thrxee prisoners who...,This a fantastic movie of three prisoners who ...,[UNK] This a fantastic movie of three prisoner...,0.519231,0.711111,0.445446,0.764364
22,"What an absolutely sDtuWnning movie, if you ha...","What an absolutely stunning movie, if you have...","[UNK] What an absolutely stunning movie, if yo...",0.560000,0.854839,0.433040,0.839547
46,Protocol is an implausible movie whose only sa...,Protocol is an implausible movie whose only sa...,[UNK] 1830 is an implausible movie whose only ...,0.657895,0.876923,0.648551,0.905022
53,I ca9nnot believe I enjoyed this as much as I ...,I cannot believe I enjoyed this as much as I d...,[UNK] I cannot believe I enjoyed this as much ...,0.627907,0.666667,0.536840,0.609072
62,So let ' s begimn!) )) The movie itself is as...,So let's begin!)))The movie itself is as origi...,[UNK] So let's sent! ) ) The movie itself is a...,0.525641,0.723077,0.489262,0.765121
63,"Btes!ides being boring, the scenes were oppres...","Besides being boring, the scenes were oppressi...","[UNK] Actors being boring, the scenes were opp...",0.526316,0.909091,0.355440,0.922045
84,"We brought this film as a joke for a friend, a...","We brought this film as a joke for a friend, a...",[UNK] We brought this film as a joke for a fri...,0.687500,0.827586,0.718032,0.856992
112,It ' s terrific when a funny movie doesn ' t m...,It's terrific when a funny movie doesn't make ...,[UNK] It's terrific when a funny movie doesn't...,0.352941,0.666667,0.245640,0.613900


In [51]:
val_100.to_csv('val_100_random_augumented.csv',index=False)

In [ ]:
val_100.shape

In [ ]:
val_100[val_100.WRR_2 > val_100.WRR_1]

In [ ]:
val_100[val_100.WRR_2 > val_100.WRR_1].shape

In [ ]:
val_100[val_100.BLEU_2 > val_100.BLEU_1].shape

In [ ]:
val_100[val_100.BLEU_2 > val_100.BLEU_1]